In [11]:
from monty.serialization import loadfn
from glob import glob
import time
from tqdm import tqdm
import collections
from PyAstronomy import pyasl
import numpy as np

In [9]:
from radqm9_pipeline.modules import bucket_mol_id,
                                    train_val_test_split,
                                    build_manager,
                                    merge_data,
                                    force_magnitude_filter,
                                    flatten,
                                    tagger
from radqm9_pipeline.elements import read_elements


# Set elements


In [5]:
elements_dict = read_elements('/pscratch/sd/m/mavaylon/sam_ldrd/radqm9_pipeline/src/radqm9_pipeline/modules/elements.pkl')

# Set Data Directory

In [13]:
base_dir = "/pscratch/sd/m/mavaylon/sam_ldrd/radqm9/updated_rad_qm9_raw/data/20230907_radQM9_trajectories"

# Load and Merge Data

In [7]:
merged_data=merge_data(base_dir)

100%|██████████| 25/25 [27:29<00:00, 65.98s/it]


# Filter Data

In [8]:
i,bucket=bucket_mol_id(merged_data)

100%|██████████| 133882/133882 [00:41<00:00, 3189.58it/s]


In [12]:
force_filtered, filtered_data = force_magnitude_filter(cutoff=10.0, data=bucket)

  0%|          | 0/130170 [00:00<?, ?it/s]


NameError: name 'np' is not defined

# Split the Data

In [6]:
data_dict_spit, weight_dist = train_val_test_split(b, train_size=.6, val_size=.2)

100%|██████████| 1132/1132 [00:00<00:00, 19784.95it/s]


TypeError: unhashable type: 'list'

# Use the BuildManager to Tag and Prepare the Data

In [27]:
prepped_data=build_manager(data_dict_spit, weight_dist)

100%|██████████| 25987/25987 [00:11<00:00, 2292.47it/s]


# Write Dataset

In [ ]:
create_dataset(prepped_data, 'rad_qm9_60_20_20', '/pscratch/sd/m/mavaylon/sam_ldrd/rad_qm9_60_20_20')